### This dataset consists of 'circles' (or 'friends lists') from Facebook. Facebook data was collected from survey participants using this Facebook app. The dataset includes node features (profiles), circles, and ego networks.

http://snap.stanford.edu/data/egonets-Facebook.html

In [4]:
#setup
from graph_tool.all import * 
import numpy as np
import math
import time

file_path = "/home/jawa/Datasets/facebook_combined.txt"

In [5]:
# initialization
g = Graph(directed=True)

neighbours = {}
vertex_id = {}
vertex_label = {}

#this time, we watn to add a edge in each direction for running pagerank and other centrality metrics

with open(file_path, encoding="UTF-8") as f:
    for line in f:
        if line.startswith("#"):
            continue
        l = line.replace("\n", "").split(" ")
        a = l[0]
        b = l[1]

        # get the vertexes
        if a not in vertex_label:
            v_a = g.add_vertex()
            vertex_label[a] = v_a
            vertex_id[g.vertex_index[v_a]] = a
        else:
            v_a = g.vertex(vertex_label[a])
        if b not in vertex_label:
            v_b = g.add_vertex()
            vertex_label[b] = v_b
            vertex_id[g.vertex_index[v_b]] = b
        else:
            v_b = g.vertex(vertex_label[b])
        # add the edges
        g.add_edge(v_a, v_b)
    

In [6]:
# degree stats
degrre_data = {}
degrre_data["out"] = []
degrre_data["in"] = []
counter = 0
for v in g.vertices():
        counter += 1
        degrre_data["out"].append(v.out_degree())
        degrre_data["in"].append(v.in_degree())
    
max_out = max(degrre_data["out"])
min_out = min(degrre_data["out"])
average_out = np.average(degrre_data["out"])
sd_out = np.std(degrre_data["out"])
print("Out degree stats -> max value = " + str(max_out) + " / min value = " + str(min_out) + " / average = " + str(average_out) + " / SV = " + str(sd_out))

Out degree stats -> max value = 1043 / min value = 0 / average = 21.8455063134 / SV = 38.8993202652


In [7]:
# distance stats
#shortest_distance = graph_tool.topology.shortest_distance(g)
#global_avg = []
#for dist in shortest_distance:
#    global_avg.append(np.average(dist))
pseudo_diameter = graph_tool.topology.pseudo_diameter(g)

# there are non connect components in the graph, so we cant just calculate the avverage shortest distance
#print("Shortest Distance Average = " + str(np.average(global_avg)) + " / Diameter = " + str(pseudo_diameter))
print("Diameter = " + str(pseudo_diameter))

Diameter = (5.0, (<Vertex object with index '0' at 0x7fb358083c78>, <Vertex object with index '4035' at 0x7fb358083e58>))


In [8]:
# clustering stats
global_clustering = graph_tool.clustering.global_clustering(g)

print ("Clustering Average = " + str(np.average(global_clustering)) + " / " + "Clustering SV = " + str(np.std(global_clustering)))

Clustering Average = 0.278343743128 / Clustering SV = 0.240830534415


In [9]:
# connected componet stats
comp, hist = label_components(g)

labels = list(set(comp.a))
largest = label_largest_component(g)
l = GraphView(g, vfilt=largest)

print("Connected components = " + str(len(labels)) + " / Edges in Largest component = " + str(l.num_edges()) + " / Vertices in Largest component = " + str(l.num_vertices()))

Connected components = 4039 / Edges in Largest component = 0 / Vertices in Largest component = 1


In [10]:
start = time.time()
# vertex closeness stats
g_closeness = closeness(g)
c_list = []
for vc in g_closeness.a:
    if not math.isnan(vc):
        c_list.append(vc)

end = time.time()
print("Time Taken: " + str(end - start) +"s")
print("Closeness stats -> max value = " + str(max(c_list)) + " / min value = " + str(min(c_list)) + " / average = " + str(np.average(c_list)) + " / SV = " + str(np.std(c_list)))

closeness_dic   = {}
zero_closeness = 0
for i in range(0, len(g_closeness.a)):
    if (g.vertex(i).out_degree() != 0):
        closeness_dic[g.vertex_index[i]] = g_closeness.a[i]
    else:
        zero_closeness += 1

Time Taken: 0.49744653701782227s
Closeness stats -> max value = 1.0 / min value = 0.113237639553 / average = 0.424170603274 / SV = 0.223657127729


In [11]:
# betweenness stats
start=time.time()
g_betweenness, edge_betweenness = betweenness(g)
v_b = list(g_betweenness)
end = time.time()
print("Time Taken: " + str(end - start) +"s")
print("Betweenness stats -> max value = " + str(max(v_b)) + " / min value = " + str(min(v_b)) + " / average = " + str(np.average(v_b)) + " / SV = " + str(np.std(v_b)))

betweenness_dic   = {}
zero_betweenness = 0
for i in range(0, len(g_betweenness.a)):
    if (g.vertex(i).out_degree() != 0):
        betweenness_dic[g.vertex_index[i]] = g_betweenness.a[i]
    else:
        zero_betweenness += 1

Time Taken: 0.3327329158782959s
Betweenness stats -> max value = 0.03299985344063619 / min value = 0.0 / average = 0.000127145013489 / SV = 0.000953272857455


In [12]:
start=time.time()
# degree centrality metrics
out_dic = {}
in_dic  = {}
zero_out = 0
zero_in  = 0
v_num = len(vertex_id.keys())
for v in g.vertices():
    if (v.out_degree() == 0):
        zero_out += 1
    else:
        out_dic[g.vertex_index[v]] = 1.0* v.out_degree() / (v_num - 1)
    if (v.in_degree() == 0):
        zero_in += 1
    else:
        in_dic[g.vertex_index[v]]  = v.in_degree() / (v_num - 1)
end = time.time()
print("Time Taken: " + str(end - start) +"s")

Time Taken: 0.06796121597290039s


In [13]:
start=time.time()
# katz
g_katz = graph_tool.all.katz(g)
katz_dic = {}
for i in range(0, len(g_katz.a)):
    katz_dic[g.vertex_index[i]] = g_katz.a[i]
end = time.time()
print("Time Taken: " + str(end - start) +"s")

Time Taken: 0.2946043014526367s


In [14]:
start=time.time()
# pagerank
g_pagerank = pagerank(g)
pagerank_dic = {}
for i in range(0, len(g_pagerank.a)):
    pagerank_dic[g.vertex_index[i]] = g_pagerank.a[i]
end = time.time()
print("Time Taken: " + str(end - start) +"s")

Time Taken: 0.32253122329711914s


In [15]:
# print centrality metrics
def topTen(metric_name, dic, string=""):
    most = {}
    less = {}
    for i in range(0, 10):
        # +
        n = max(dic, key=dic.get)
        most[n] = dic[n]
        del dic[n]
        # -
        n = min(dic, key=dic.get)
        less[n] = dic[n]
        del dic[n]    
    print(metric_name)
    print(string)
    print("Most central:")
    print(sorted(most.items(), key=lambda x: x[1]))
    print("Least central:")
    print(sorted(less.items(), key=lambda x: x[1]))
    print("="*100)
    
topTen("Degree centrality (IN)", in_dic, "Total zeros: "+str(zero_in))
topTen("Degree centrality (OUT)", out_dic, "Total zeros: "+str(zero_out))
topTen("Betweenness", betweenness_dic, "Total zeros: "+str(zero_betweenness))
topTen("Closeness", closeness_dic, "Total zeros: "+str(zero_closeness))
topTen("Katz", katz_dic)
topTen("PageRank", pagerank_dic)

Degree centrality (IN)
Total zeros: 2
Most central:
[(3439, 0.04507181773155027), (1318, 0.04507181773155027), (3441, 0.04507181773155027), (3443, 0.045319465081723624), (1215, 0.045319465081723624), (1312, 0.04606240713224369), (3445, 0.04878652798415057), (1285, 0.05349182763744428), (1490, 0.06092124814264487), (1373, 0.06215948489351164)]
Least central:
[(1, 0.00024764735017335313), (2, 0.00024764735017335313), (3, 0.00024764735017335313), (4, 0.00024764735017335313), (5, 0.00024764735017335313), (6, 0.00024764735017335313), (7, 0.00024764735017335313), (8, 0.00024764735017335313), (10, 0.00024764735017335313), (11, 0.00024764735017335313)]
Degree centrality (OUT)
Total zeros: 376
Most central:
[(2944, 0.046805349182763745), (366, 0.04730064388311045), (2995, 0.0512630014858841), (2126, 0.053244180287270924), (348, 0.05572065378900446), (0, 0.08593363051015354), (1821, 0.1342248637939574), (352, 0.18524021792966816), (351, 0.19266963843486876), (107, 0.25829618623080736)]
Least cen